<a href="https://colab.research.google.com/github/djrl10/trying/blob/master/lstm_okay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [370]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [371]:
import tensorflow as tf
import pandas as pd
from io import StringIO
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import os
import glob
import re , itertools
from html import unescape
!pip install unidecode
from unidecode import unidecode
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from string import punctuation

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.activations import elu, softmax

from sklearn.metrics import confusion_matrix

import seaborn

from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import LSTM

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [372]:
df = pd.read_csv("/content/drive/My Drive/datadataset/KC1.csv")

In [373]:
df.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
       'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
       'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 'class'],
      dtype='object')

In [374]:
df.dtypes

loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                    float64
d                    float64
i                    float64
e                    float64
b                    float64
t                    float64
lOCode                 int64
lOComment              int64
lOBlank                int64
locCodeAndComment      int64
uniq_Op              float64
uniq_Opnd            float64
total_Op             float64
total_Opnd           float64
branchCount          float64
class                   bool
dtype: object

In [375]:
df.shape

(2109, 22)

In [376]:
df.isnull().sum()

loc                  0
v(g)                 0
ev(g)                0
iv(g)                0
n                    0
v                    0
l                    0
d                    0
i                    0
e                    0
b                    0
t                    0
lOCode               0
lOComment            0
lOBlank              0
locCodeAndComment    0
uniq_Op              0
uniq_Opnd            0
total_Op             0
total_Opnd           0
branchCount          0
class                0
dtype: int64

In [377]:
df.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,class
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,0.31,1187.70,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,0.26,635.37,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,0.08,132.33,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True


In [378]:
# One-hot encoding
classification = pd.get_dummies(df[df.columns[1]])
print(classification.shape)

(2109, 31)


In [379]:
X = df.drop(["loc", "branchCount", "class"],axis=1)   # drop the target variable from df

In [380]:
X.head()

,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd
0,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2
1,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1.0,1.0,1.0,1.0
2,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,0.31,1187.70,65,10,6,0,18.0,25.0,107.0,64.0
3,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,0.26,635.37,37,2,5,0,16.0,28.0,89.0,52.0
4,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,0.08,132.33,21,0,2,0,11.0,10.0,41.0,17.0


In [381]:
y = df[["class"]]

In [382]:
y.head()

,class
0,False
1,True
2,True
3,True
4,True


#split the data

In [383]:
from sklearn.model_selection import train_test_split

In [384]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

#Size of train data

In [385]:
x_train.shape  , y_train.shape

((1476, 19), (1476, 1))

#Size of test data

In [386]:
x_test.shape , y_test.shape

((633, 19), (633, 1))

#model

In [387]:
# Architecture
n_input = x_train.shape[1]
n_hidden1 = 30
n_hidden2 = 20
n_hidden3 = 15
n_output = y_train.shape[1]

# set dropout probability
dropout_prob = 0.1

In [388]:
model = tf.keras.models.Sequential([
    Dense(n_input, input_shape = (n_input,), activation = elu),   # Input layer
    
    Dense(n_hidden1, activation = elu), # hidden layer 1
    Dropout(dropout_prob),     
    
    Dense(n_hidden2, activation = elu), # hidden layer 2
    Dropout(dropout_prob), 
    
    Dense(n_hidden3, activation = elu), # hidden layer 3
    Dropout(dropout_prob), 
    
    Dense(n_output, activation = softmax)  # Output layer

])

In [389]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 19)                380       
_________________________________________________________________
dense_63 (Dense)             (None, 30)                600       
_________________________________________________________________
dropout_36 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 20)                620       
_________________________________________________________________
dropout_37 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 15)                315       
_________________________________________________________________
dropout_38 (Dropout)         (None, 15)              

#lstm

In [390]:
# define and fit the model
def get_model(x_train, y_train):
	# define model
    model.add(LSTM(50, dropout=0.0, recurrent_dropout=0.0,input_shape=(None, 1)))
    model.add(Dense(100, input_dim=2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

  #compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

  # fit model
    model.fit(x_train, y_train)
    return model

  # fit model
    model = get_model(x_train, y_train) 

# predict probabilities for test set
yhat_probs = model.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [391]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.154818
Precision: 0.154818
Recall: 1.000000
F1 score: 0.268126


#SVM

In [392]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [393]:
svc_model = SVC()

In [394]:
svc_model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [395]:
svc_pred = svc_model.predict(x_test)

In [396]:
accuracy = accuracy_score(svc_pred,y_test)

precision = precision_score(svc_pred, y_test)

recall = recall_score(svc_pred, y_test)

f1 = f1_score(svc_pred, y_test)

In [397]:
accuracy

0.8499210110584519

In [398]:
precision

0.15306122448979592

In [399]:
recall

0.5555555555555556

In [400]:
f1

0.24